# How to Use the Assistants API to Build a Math Chatbot with a Code Interpreter

## Introduction

Welcome to this tutorial on utilizing the OpenAI Assistants API to build an interactive Math Chatbot with a Code Interpreter feature.

The OpenAI Assistants API enables developers to create sophisticated AI-powered assistants within applications. It offers tools such as **Code Interpreter**, **Retrieval**, and **Function Calling**. The API supports models like GPT-3.5 or GPT-4 and can be integrated into various applications for use cases including mentoring, customer support, and more.

In this tutorial, we will explore how to use the Assistants API with the **Code Interpreter** to build a math mentoring chatbot. This chatbot will engage users conversationally and automatically interpret and execute code snippets for math problems, making it a versatile learning and problem-solving tool in mathematics.

## Setting Up the Environment

First, install the necessary Python packages. Run the following command to install OpenAI and Gradio:

In [ ]:
!pip install openai gradio

Then, provide your OpenAI API key.

In [4]:
import os

os.environ['OPENAI_API_KEY'] = 'your_api_key_here'

## Creating an Assistant

We will create an Assistant to manage interactions with the OpenAI Assistants API. An Assistant is an entity that can be configured to respond to user messages.

In [78]:
import openai

client = openai.Client()

def create_assistant():
  return client.beta.assistants.create(
    name="Math Mentor",
    instructions="You are a supportive and engaging math mentor for school children, utilizing a code interpreter for accurate problem-solving. Simplify complex concepts with age-appropriate language and examples, ensuring content alignment with educational standards while maintaining child safety and privacy.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106" # Can find out all models at https://openai.com/pricing
  )


assistant = create_assistant()

Large language models (LLMs) like GPT-4 have impressive capabilities across a broad range of tasks, such as predicting the next word. However, they have limitations, such as lack of precision in computation and handling complex calculations. Using a code interpreter or additional tools can be particularly beneficial in these cases. To demonstrate the use of the Code Interpreter, we are creating an Assistant as a Math Mentor.

### Creating a Thread
A Thread represents an ongoing conversation with a user.

In [79]:
client = openai.Client()
thread = client.beta.threads.create()

## Sending a User Message

To send a user message, create a message containing user queries and possibly files.

Then, create a Run. The Assistant reads the Thread and decides whether to call tools, e.g., Code Interpreter, or use the model to answer.

Periodically retrieve the Run to check its status and see if it has moved to `completed` or `failed`.

Once it is `completed`, you can `yield` the `'assistant'` message.

In [84]:
import time

def ask_assistant(assistant, thread_id, question):
  client.beta.threads.messages.create(
      thread_id=thread_id,
      role="user",
      content=question
  )

  run = client.beta.threads.runs.create(
      thread_id=thread_id,
      assistant_id=assistant.id
  )

  print("retrieving data...")
  while True:
    # Retrieve the current status of the Run
    run_status = client.beta.threads.runs.retrieve(
      thread_id=thread_id,
      run_id=run.id
    )

    # Check if the Run is completed
    print(f"run status: {run_status.status}")
    if run_status.status == 'completed':
        print("Run is completed")
        break
    elif run_status.status == 'failed':
        yield f"Error occurred!"
        return  # Exit the function after yielding the error message

    # If not completed, wait for some time before next check
    time.sleep(1)  # Waits for 1 second before the next check

  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)

  for message in messages.data:
    # Check if the message belongs to the current run, and it's assistant message
    if message.run_id == run.id and message.role == 'assistant':
        for content in message.content:
            yield content.text.value

Test this method and print out the `yield`ed messages.

In [86]:
for message in ask_assistant(assistant, thread.id, "I need to solve the equation 3x + 11 = 14. Can you help me?"):
  print(message)

retrieving data...
run status: queued
run status: in_progress
run status: in_progress
run status: in_progress
run status: in_progress
run status: in_progress
run status: in_progress
run status: in_progress
run status: in_progress
run status: completed
Run is completed
SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_JvU65Egnn89RpFi7s9n23vnk', assistant_id='asst_4PUSw6z2aWKoXUG0KVWPuGLP', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=1699599855, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_SiPo84eL1cyVsiYnscAJ0S36', thread_id='thread_7ICt9Lzwf23CRGFZqRRDYWoo'), ThreadMessage(id='msg_hG5V7MoIZv2jGMGrSSl3FtGU', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation 3x + 11 = 14. Can you help me?'), type='text')], created_at=1699599845, file_ids=[], metadata={}, object='thread.message', role='user'

## Integrating Gradio for a Web Interface

Gradio is a Python library that simplifies creating UIs for machine learning models, ideal for quickly deploying models as web applications.

We will use `ChatInterface` to create a Chat Web UI.

In [85]:
def chat_with_math_mentor(user_input, history):
  # Chat with AI with defined assistant and thread
  yield from ask_assistant(assistant, thread.id, user_input)


In [88]:
import gradio as gr

chat = gr.ChatInterface(
    chat_with_math_mentor,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="I can help you with math questions", container=False, scale=7),
    title="Math Mentor",
    description="Ask Math Mentor any math question",
    theme="soft",
    examples=["I need to solve the equation 3x + 11 = 14. Can you help me?", "3 ** 4", "What's linear regression?"],
  ).queue()

chat.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6f1a6b282e600408fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Conclusion and Further Learning

Congratulations on completing this tutorial! You now have a solid understanding of how to use the OpenAI Assistants API with the Code Interpreter to create an intelligent math chatbot.

For further learning, explore these resources:
- [OpenAI API Documentation](https://platform.openai.com/docs/assistants/overview)
- [Gradio Documentation](https://gradio.app/docs/)

Please check out the runnable Notebook on [Kaggle](https://www.kaggle.com/code/jakelinai/use-openai-assistants-api-with-code-interpreter).